In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last" # all | last | last_expr | none 

In [3]:
# for name in dir():
#     if not name.startswith('_'):
#         del globals()[name]

In [4]:
# ============= Import required packaages | Define global variables ==============

# Import all custom variables and modules
from custom_classes_defs.preprocessing import *
from custom_classes_defs.hed import *

RND_STATE = 247
BATCH_SIZE = 256
EPOCHS = 100
INTERACTIVE_SESSION = True

FREEZE = 0

keras.utils.set_random_seed(RND_STATE)
from keras.utils import plot_model

# -------------------------------------------------------


In [ ]:
# Verify tensorflow/keras versions
print(f"tensorflow version: {tf.__version__}")
print(f"keras version: {keras.__version__}")

# Verify CPU/GPU availability
print(tf.config.list_physical_devices())
NUM_GPU = len(tf.config.list_physical_devices('GPU'))
print(f"Number of GPUs assigned for computation: {NUM_GPU}")

if NUM_GPU:
    # print GPU info
    !nvidia-smi
    
if NUM_GPU>1:
    strategy = tf.distribute.MirroredStrategy()
    BATCH_SIZE = strategy.num_replicas_in_sync * BATCH_SIZE

### Data preparation and model configurations

In [ ]:
print("{}\n\t{}\n{}".format('='*55,'Data preparation and model configurations', '-'*55))
# Images and annations for Thebe seismic data
start = time.time()
img_url = '../thebe_new/seismic'
target_url = '../thebe_new/fault'
seis = Thebe(img_url, target_url)
    
# Create datasets for respective data samples
train_dataset = seis.data_generator('train', 
    batch_size=BATCH_SIZE, cache=NUM_GPU, weighted_loss=True
) 
val_dataset = seis.data_generator('val', 
    batch_size=BATCH_SIZE, cache=NUM_GPU, weighted_loss=True
)
x_test, y_test = seis.data_generator('test', as_numpy=True)
        
print("Train Dataset:", train_dataset)

print("Size of training data: {}".format(seis.train_size))
print("Size of validation data: {}".format(seis.validation_size))
print("Size of test data: {}".format(seis.test_size)) 
print("Data images tensor:",train_dataset.element_spec[0])
print("Data labels tensor:",train_dataset.element_spec[1])
print("Data weights tensor:",train_dataset.element_spec[-1])

print('...elapsed time: ___{:5.2f} minutes___'.format((time.time()-start) / 60))

In [ ]:
if INTERACTIVE_SESSION:
    # a = next(train_dataset.as_numpy_iterator())
    # a = next(val_dataset.as_numpy_iterator())
    # seis.display_sample_images(a[0], a[1], num_images=4)
    # seis.display_sample_images(a[0], a[1], num_images=4)

    seis.display_sample_images(x_test, y_test, num_images=4)
    seis.display_sample_images(x_test, y_test, num_images=4)


In [ ]:
# Model configurations
conf = model_config(
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    shuffle=True,
    scaling=1,
    save_path=f'./Thebe/hed{FREEZE}',
    img_shape=seis.img_size,
    target_size=seis.img_size,
    threshold=seis.threshold,
    pos_label=seis.pos_label,
    train_size=seis.train_size,
    test_size=seis.test_size,
    new_training_session=True,
    multiple_gpu_device=(NUM_GPU>1),
    validation_size=seis.validation_size
)

out_no = 5
hed_out = f'output{out_no}_'
callbacks = conf.callbacks(
    chkpt_monitor=f'val_{hed_out}f1_score', 
    es_monitor=f'{hed_out}accuracy',
    es_patience=EPOCHS // 4, 
    lr_monitor=f'val_{hed_out}loss'
)

conf.set( validation_data=val_dataset,  callbacks=callbacks )

def compilation_step():
    m1 = f1_score(positive_label=seis.pos_label, threshold=seis.threshold)
    weighted_dict = dict([(f'output{k}','binary_crossentropy') for k in range(5)])
    conf.set(
        'compile', 
        metrics= ['accuracy', m1],
        weighted_metrics=weighted_dict
    )

# conf.double_check(INTERACTIVE_SESSION)
conf.info()

### Build  model 

In [ ]:
print("\n\n{}\n\t{}\n{}".format('='*55,f'Build model', '-'*55))

if conf.multiple_gpu_device:

    # strategy = tf.distribute.MirroredStrategy()
    print(f"Number of devices: {strategy.num_replicas_in_sync}")
    with strategy.scope():

        compilation_step()
        m_obj = HED2D(model_arch=conf.model_arch, num_freeze=FREEZE)
        model = m_obj.build_model()
        model.compile(**conf.compile_args)

else:

    compilation_step()
    m_obj = HED2D(model_arch=conf.model_arch, num_freeze=FREEZE)
    model = m_obj.build_model()
    model.compile(**conf.compile_args)


model.summary()
# keras.utils.plot_model(model, 'm_obj.png',show_shapes=True)
# plot_model(model, 'm_obj.png',show_shapes=True)
num_trainable_weights = sum([np.prod(w.shape) for w in model.trainable_weights])
print(f"Total number of parameters: {model.count_params():,}")
print(f"Total trainable wieghts: {num_trainable_weights:,}")
print(f"Total non-trainable wieghts: {model.count_params()-num_trainable_weights:,}")



### Train  model

In [ ]:
print("\n\n{}\n\t{}\n{}".format('='*55,f'Train {m_obj.Name} model', '-'*55))

model, train_history = \
    conf.execute_training(
        model, 
        data=train_dataset, 
        plot_history=INTERACTIVE_SESSION,
        metrics=[f'{hed_out}loss', f'val_{hed_out}loss'],
        save_model_history=False
)


In [ ]:
if INTERACTIVE_SESSION:
    for out_no in range(6):
        hed_out = f'output{out_no}_'
        show_convergence(train_history.history, [hed_out+'accuracy','val_'+hed_out+'accuracy'])

In [ ]:
if INTERACTIVE_SESSION:
    for out_no in range(6):
        hed_out = f'output{out_no}_'
        show_convergence(train_history.history, [hed_out+'f1_score','val_'+hed_out+'f1_score'])

In [ ]:
if INTERACTIVE_SESSION:
    for out_no in range(6):
        hed_out = f'output{out_no}_'
        show_convergence(train_history.history, [hed_out+'loss','val_'+hed_out+'loss'])

In [ ]:
if INTERACTIVE_SESSION:
    show_convergence(train_history.history, 'lr')

### Evaluate and Vizualize

In [ ]:
# print("\n\n{}\n\t{}\n{}".format('='*55,f'Evaluate {m_obj.Name} model', '-'*55))
y_pred = model.predict(x_test, batch_size=BATCH_SIZE, verbose=2)


In [ ]:
Num_outputs = len(y_pred)

if INTERACTIVE_SESSION:
    for out_no in range(Num_outputs):
        print(f"output {out_no}")
        seis.display_sample_images(x_test, y_test, y_pred[out_no])
    

In [ ]:
# model.evaluate(x=x_test, batch_size=BATCH_SIZE, verbose=2)


In [ ]:
for out_no in range(Num_outputs):
    print(f"{'='*33}\n\toutput {out_no}\n{'-'*33}")
    scores = conf.evaluate_sklearn(y_test, y_pred[out_no],report=True)
    print(scores)